# PriceTrack: Unlocking Bike Market Insights

PriceTrack is a data science project designed to predict the valuation of used bike based on key input parameters.
Leveraging Multiple Linear regression model, it provides data-driven insights to help sellers make informed decisions.

## Developing Regression Model

This step involves building and training regression model, encoding features, and evaluating performance using metrics like R², MAE, MSE, and RMSE to ensure accurate predictions.

- For our project, we have used Multiple Linear Regression since output feature is a continuous variable and we want to predict it.

- To handle categorical data, we have used one hot encoding.

Importing all the required modules and functions

In [98]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

Loading the cleaned data

In [99]:
df = pd.read_csv("Cleaned_Bike_Data.csv")
df

,model,price,city,kms_driven,owner,age,power,brand,owner_encoded
0,TVS Star City Plus Dual Tone 110cc,35000,Ahmedabad,17654,First Owner,3,110,TVS,1
1,Royal Enfield Classic 350cc,119900,Delhi,11000,First Owner,4,350,Royal Enfield,1
2,Triumph Daytona 675R,600000,Delhi,110,First Owner,8,675,Triumph,1
3,TVS Apache RTR 180cc,65000,Bangalore,16329,First Owner,4,180,TVS,1
4,Yamaha FZ S V 2.0 150cc-Ltd. Edition,80000,Bangalore,10000,First Owner,3,150,Yamaha,1
...,...,...,...,...,...,...,...,...,...
32518,Hero Passion Pro 100cc,39000,Delhi,22000,First Owner,4,100,Hero,1
32519,TVS Apache RTR 180cc,30000,Karnal,6639,First Owner,9,180,TVS,1
32520,Bajaj Avenger Street 220,60000,Delhi,20373,First Owner,6,220,Bajaj,1
32521,Hero Super Splendor 125cc,15600,Jaipur,84186,First Owner,16,125,Hero,1


### 🏍️ Defining Features and Target Variable

- **`X` (Features)**: The independent variables used for prediction:
  - `age`: Age of the bike in years (numerical).
  - `power`: Engine capacity in cc (numerical).
  - `brand`: Brand of the bike (categorical).
  - `owner_encoded`: Encoded representation of ownership status (ordinal categorical).
  - `city`: City where the bike is listed (categorical).
  - `kms_driven`: Total distance the bike has been ridden (numerical).

- **`y` (Target Variable)**:  
  - `price`: The dependent variable representing the bike's resale price.

This selection enables the model to learn how various mechanical, demographic, and usage factors influence a bike’s resale value.

In [100]:
X = df[["age", "power", "brand", "owner_encoded", "city", "kms_driven"]]
y = df["price"]

### Train-Test Split

- The dataset is split into **training** and **testing** sets to evaluate model performance.
- **`train_test_split(X, y, test_size=0.2, random_state=42)`**:
  - **80%** of the data is used for training (`X_train`, `y_train`).
  - **20%** of the data is reserved for testing (`X_test`, `y_test`).
  - `random_state=42` ensures reproducibility by generating the same split every time.

This helps in assessing how well the model generalizes to unseen data.


In [101]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

### 🔧 Data Preprocessing Pipeline – Used Bike Dataset

The `preprocessor` is a `ColumnTransformer` that applies suitable transformations to different feature types:

- **One-Hot Encoding (`OneHotEncoder`)**: Applied to categorical variables (`brand`, `city`) to convert them into a machine-readable numerical format, while handling unknown categories gracefully.
- **Feature Scaling (`StandardScaler`)**: Applied to numerical features (`age`, `power`, `kms_driven`) to standardize their values (zero mean and unit variance), improving model stability and convergence.
- **Pass-Through**: The `owner_encoded` feature (already numeric and ordinal) is passed without transformation.

This pipeline ensures consistent preprocessing across diverse data types, making the data well-prepared for regression modeling.

In [102]:
preprocessor = ColumnTransformer(
    [
        ("onehot", OneHotEncoder(handle_unknown="ignore"), ["brand", "city"]),
        ("scaler", StandardScaler(), ["age", "power", "kms_driven"]),
    ],
    remainder="passthrough",  # Keeps 'owner_encoded'
    force_int_remainder_cols=False,  # 👈 Enables future behavior now
)

preprocessor

ColumnTransformer(force_int_remainder_cols=False, remainder='passthrough',
                  transformers=[('onehot',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['brand', 'city']),
                                ('scaler', StandardScaler(),
                                 ['age', 'power', 'kms_driven'])])

### Model Pipeline

A **`Pipeline`** is used to streamline preprocessing and model training in a single workflow:

- **`preprocessor`**: Applies transformations to the input data (One-Hot Encoding & Standard Scaling).
- **`LinearRegression()`**: The regression model that learns the relationship between the features and the target variable.

This approach ensures that preprocessing steps are consistently applied during both training and prediction, improving efficiency and reducing the risk of data leakage.

In [103]:
model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", LinearRegression())
])

# Train Model
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(force_int_remainder_cols=False,
                                   remainder='passthrough',
                                   transformers=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['brand', 'city']),
                                                 ('scaler', StandardScaler(),
                                                  ['age', 'power',
                                                   'kms_driven'])])),
                ('regressor', LinearRegression())])

### Model Evaluation Metrics

- **Mean Absolute Error (MAE)**: Measures the average absolute difference between actual and predicted prices. Lower values indicate better accuracy.
- **Mean Squared Error (MSE)**: Similar to MAE but gives higher weight to larger errors, making it more sensitive to outliers.
- **Root Mean Squared Error (RMSE)**: Square root of MSE, providing an interpretable error measure in the same unit as price.
- **R² Score**: Indicates how well the model explains the variance in price; closer to 1 means a better fit.

In [104]:
# Predictions
y_pred = model.predict(X_test)

# Model Evaluation
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Display Metrics
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R2 Score: {r2:.2f}")

MAE: 9896.80
MSE: 635321658.66
RMSE: 25205.59
R2 Score: 0.91


In [105]:
# Checking the co-efficients

# feature_names = preprocessor.get_feature_names_out()
# coef_df = pd.DataFrame(model.named_steps["regressor"].coef_, index=feature_names, columns=["Coefficient"])
# coef_df

## Integration with Statsmodel

In [106]:
import statsmodels.api as sm

In [107]:
# Encode categorical variables
X_train_encoded = pd.get_dummies(X_train, drop_first=True)
X_test_encoded = pd.get_dummies(X_test, drop_first=True)
X_overall_encoded = pd.get_dummies(X, drop_first=True)

# Align the columns of test and overall with train
X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)
X_overall_encoded = X_overall_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)

# Add constant (intercept)
X_train_encoded = sm.add_constant(X_train_encoded)
X_test_encoded = sm.add_constant(X_test_encoded)
X_overall_encoded = sm.add_constant(X_overall_encoded)

# Force float dtype
X_train_encoded = X_train_encoded.astype(float)
X_test_encoded = X_test_encoded.astype(float)
X_overall_encoded = X_overall_encoded.astype(float)

y_train = y_train.astype(float)
y_test = y_test.astype(float)
y_overall = y.astype(float)

# Fit the model
model_sm = sm.OLS(y_train, X_train_encoded).fit()

# Predictions
y_train_pred_sm = model_sm.predict(X_train_encoded)
y_test_pred_sm = model_sm.predict(X_test_encoded)
y_overall_pred_sm = model_sm.predict(X_overall_encoded)

results = {
    "r2": [
        model_sm.rsquared,
        r2_score(y_test, y_test_pred_sm),
        r2_score(y_overall, y_overall_pred_sm),
    ],
    "mae": [
        mean_absolute_error(y_train, y_train_pred_sm),
        mean_absolute_error(y_test, y_test_pred_sm),
        mean_absolute_error(y_overall, y_overall_pred_sm),
    ],
    "mse": [
        mean_squared_error(y_train, y_train_pred_sm),
        mean_squared_error(y_test, y_test_pred_sm),
        mean_squared_error(y_overall, y_overall_pred_sm),
    ],
}

results["rmse"] = np.sqrt(results["mse"])
results["r2_percent"] = [f"{val*100:.0f}%" for val in results["r2"]]

# Print Metrics
result_df = pd.DataFrame(
    {
        "Percentage Accuracy": results["r2_percent"],
        "R² Score": results["r2"],
        "MAE": results["mae"],
        "MSE": results["mse"],
        "RMSE": results["rmse"]
    },
    index=["Training", "Testing", "Overall"],
)
# result_df[["R² Score","MAE", "MSE", "RMSE"]].agg(lambda s: ['%.2f'%val for val in s]) # Formatting
result_df

,Percentage Accuracy,R² Score,MAE,MSE,RMSE
Training,92%,0.917816,9391.708345,6.945422e+08,26354.169022
Testing,91%,0.914907,9892.261257,6.348624e+08,25196.476682
Overall,92%,0.917291,9491.825084,6.826055e+08,26126.720676


### Statsmodel Summary on Testing Data

In [108]:
model_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.918
Model:                            OLS   Adj. R-squared:                  0.916
Method:                 Least Squares   F-statistic:                     666.1
Date:                Wed, 30 Apr 2025   Prob (F-statistic):               0.00
Time:                        11:25:27   Log-Likelihood:            -3.0177e+05
No. Observations:               26018   AIC:                         6.044e+05
Df Residuals:                   25588   BIC:                         6.079e+05
Df Model:                         429                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                      4.292e+05   1.12e+04     38.237      0.000    4.07e+05    4.51e+05
age                       -2211.8795     76.750    -28.819      0.000   -2362.314   -2061.445
power                       515.4707      3.297    156.340      0.000     509.008     521.933
owner_encoded             -1.206e+04    716.497    -16.832      0.000   -1.35e+04   -1.07e+04
kms_driven                   -0.4164      0.014    -29.068      0.000      -0.444      -0.388
brand_Bajaj               -4.334e+05   7306.982    -59.310      0.000   -4.48e+05   -4.19e+05
brand_Benelli             -2.946e+05   8313.829    -35.438      0.000   -3.11e+05   -2.78e+05
brand_Ducati               6.693e+04   9801.978      6.828      0.000    4.77e+04    8.61e+04
brand_Harley-Davidson       -3.4e+05   7807.039    -43.555      0.000   -3.55e+05   -3.25e+05
brand_Hero                -4.116e+05   7355.289    -55.959      0.000   -4.26e+05   -3.97e+05
brand_Honda                -4.11e+05   7348.670    -55.934      0.000   -4.25e+05   -3.97e+05
brand_Hyosung             -3.485e+05   8264.117    -42.172      0.000   -3.65e+05   -3.32e+05
brand_Indian               1.084e+04   1.74e+04      0.623      0.534   -2.33e+04     4.5e+04
brand_Jawa                -3.735e+05   1.19e+04    -31.434      0.000   -3.97e+05    -3.5e+05
brand_KTM                 -3.975e+05   7317.178    -54.324      0.000   -4.12e+05   -3.83e+05
brand_Kawasaki            -2.842e+05   8132.876    -34.948      0.000      -3e+05   -2.68e+05
brand_LML                 -3.828e+05   2.76e+04    -13.864      0.000   -4.37e+05   -3.29e+05
brand_MV                   2.673e+05   2.02e+04     13.233      0.000    2.28e+05    3.07e+05
brand_Mahindra            -4.204e+05   8710.046    -48.271      0.000   -4.38e+05   -4.03e+05
brand_Royal Enfield       -4.748e+05   7263.868    -65.368      0.000   -4.89e+05   -4.61e+05
brand_Suzuki              -4.143e+05   7381.045    -56.128      0.000   -4.29e+05      -4e+05
brand_TVS                 -4.179e+05   7411.537    -56.387      0.000   -4.32e+05   -4.03e+05
brand_Triumph             -2.155e+04   9323.517     -2.312      0.021   -3.98e+04   -3278.955
brand_Yamaha              -4.087e+05   7354.838    -55.563      0.000   -4.23e+05   -3.94e+05
city_Abohar               -6264.6563   2.06e+04     -0.304      0.761   -4.66e+04    3.41e+04
city_Adalaj               -2907.7992   1.57e+04     -0.185      0.853   -3.37e+04    2.79e+04
city_Adoni                 3.988e+04   2.79e+04      1.430      0.153   -1.48e+04    9.45e+04
city_Adyar                 1.822e+04   1.75e+04      1.041      0.298   -1.61e+04    5.25e+04
city_Agra                 -3058.8172   1.26e+04     -0.242      0.808   -2.78e+04    2.17e+04
city_Ahmedabad            -1704.2742   8503.786     -0.200      0.841   -1.

### 📊 Model Testing: Manual Test Cases for Used Bikes

To validate model behavior, we manually test different scenarios based on bike attributes.

| **Brand**       | **City**     | **KMs Driven** | **Age (Years)** | **Power (cc)** | **Owner (Encoded)** | **Description**                          | **Expected Outcome**                                |
|-----------------|--------------|----------------|------------------|----------------|---------------------|------------------------------------------|------------------------------------------------------|
| Yamaha          | Bangalore    | 5,000          | 1                | 150            | 1                   | New bike with low mileage and trusted brand | High predicted price due to condition and brand     |
| Royal Enfield   | Pune         | 70,000         | 5                | 350            | 1                   | Mid-aged popular cruiser                  | Moderate to high price for brand and engine         |
| Hero            | Patna        | 120,000        | 10               | 100            | 2                   | Old budget bike, second owner             | Low predicted price due to age and condition        |
| Triumph         | Mumbai       | 15,000         | 2                | 675            | 1                   | Premium sports bike with low mileage      | Very high predicted price due to luxury brand       |
| Bajaj           | Jaipur       | 80,000         | 7                | 150            | 3                   | Mid-age bike, 3rd owner                   | Lower price due to high ownership and wear          |

### **Key Trends Expected**
- **Newer bikes with low mileage** → **Higher price**  
- **Older bikes or with high ownership** → **Lower price**  
- **Luxury brands (e.g., Triumph)** → **High resale value**  
- **Popular commuter brands (e.g., Hero, Bajaj)** → **Economical resale values**


In [109]:
# Define test cases as a list of dictionaries
test_cases = [
    {
        "brand": "Yamaha",
        "city": "Bangalore",
        "kms_driven": 5000,
        "age": 1,
        "power": 150,
        "owner_encoded": 1,
        "Description": "New bike with low mileage and trusted brand",
    },
    {
        "brand": "Royal Enfield",
        "city": "Pune",
        "kms_driven": 70000,
        "age": 5,
        "power": 350,
        "owner_encoded": 1,
        "Description": "Mid-aged popular cruiser",
    },
    {
        "brand": "Hero",
        "city": "Patna",
        "kms_driven": 120000,
        "age": 10,
        "power": 100,
        "owner_encoded": 2,
        "Description": "Old budget bike, second owner",
    },
    {
        "brand": "Triumph",
        "city": "Mumbai",
        "kms_driven": 15000,
        "age": 2,
        "power": 675,
        "owner_encoded": 1,
        "Description": "Premium sports bike with low mileage",
    },
    {
        "brand": "Bajaj",
        "city": "Ahmedabad",
        "kms_driven": 2300,
        "age": 3,
        "power": 125,
        "owner_encoded": 1,
        "Description": "Mid-age bike, 3rd owner",
    },
]

# Convert test cases to DataFrame
test_df = pd.DataFrame(test_cases)

# Predict prices for test cases
predicted_prices = model.predict(test_df.drop(columns=["Description"]))

# Add predicted prices to DataFrame
test_df["Predicted Price (₹)"] = [f"₹{price:,.0f}" for price in predicted_prices]

# Display the results
test_df

,brand,city,kms_driven,age,power,owner_encoded,Description,Predicted Price (₹)
0,Yamaha,Bangalore,5000,1,150,1,New bike with low mileage and trusted brand,"₹72,221"
1,Royal Enfield,Pune,70000,5,350,1,Mid-aged popular cruiser,"₹89,187"
2,Hero,Patna,120000,10,100,2,"Old budget bike, second owner","₹-25,042"
3,Triumph,Mumbai,15000,2,675,1,Premium sports bike with low mileage,"₹723,049"
4,Bajaj,Ahmedabad,2300,3,125,1,"Mid-age bike, 3rd owner","₹38,876"
